In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import funcs
import imp
import random
import mne
imp.reload(funcs)

C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_18888\3916662429.py:11: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'funcs' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\intelligence_tRNS\\tRNS-Intelligence\\processing\\funcs.py'>

In [2]:
exp_info = funcs.load_info()
subjects_behavior = funcs.load_subjects_behavior()
subjects_behavior["raw"] = None

c:\Users\Tianyi Zheng\OneDrive - neuron.t.u-tokyo.ac.jp\Documents\zheng\mywork\intelligence_tRNS\tRNS-Intelligence\processing\funcs.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  subjects_behavior = pd.concat([subjects_behavior, one_behavior], ignore_index=True)


In [3]:
task_time_name = ['pre', 'post']
for subject_id in range(1, 27):
    for tasktime in range (1, 3):
        raw_path = os.path.join('..', '..', '..', 'data', str(subject_id), 'raw_' + str(subject_id) + '_' + str(tasktime) + '.fif')
        raw = mne.io.read_raw_fif(raw_path, preload=True)
        # Locate question events
        question_times = mne.find_events(raw, stim_channel='question')[:, 0]
        response_times = mne.find_events(raw, stim_channel='response')[:, 0]

        idx_start = (subject_id-1) * 82 + (tasktime-1) * 41
        idx_end = idx_start + 40
        inraw_index = 0

        for idx in range (idx_start, idx_end+1): # 41 trials
            # print("subject_id: ", subject_id, "tasktime: ", tasktime, "trial: ", inraw_index+1, "/41")
            if subjects_behavior.at[idx, "answered"] == 1:
                print("subject_id: ", subject_id, "tasktime: ", tasktime, "trial: ", inraw_index+1, "/41")
                # write here
                question_time = question_times[inraw_index]
                response_idx = np.searchsorted(response_times, question_time, side='right')
                response_time = response_times[response_idx]
                if (response_time - question_time) / raw.info['sfreq'] > 90:
                    print("\nTOO LONG!!\n")
                raw_trial = raw.copy().crop(tmin=question_time/raw.info['sfreq'], tmax=response_time/raw.info['sfreq'])
                subjects_behavior.at[idx, "raw"] = raw_trial
          
            inraw_index += 1
# 11min28.9s

Opening raw data file ..\..\..\data\1\raw_1_1.fif...
    Read a total of 1 projection items:
        EOG-eeg--0.200-0.200-PCA-01 (1 x 31) active
    Range : 0 ... 3865075 =      0.000 ...   805.224 secs
Ready.


C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_18888\3499214276.py:5: RuntimeWarning: This filename (..\..\..\data\1\raw_1_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(raw_path, preload=True)


Reading 0 ... 3865075  =      0.000 ...   805.224 secs...
41 events found on stim channel question
Event IDs: [1]
41 events found on stim channel response
Event IDs: [1]
subject_id:  1 tasktime:  1 trial:  1 /41
subject_id:  1 tasktime:  1 trial:  2 /41
subject_id:  1 tasktime:  1 trial:  3 /41
subject_id:  1 tasktime:  1 trial:  4 /41
subject_id:  1 tasktime:  1 trial:  5 /41
subject_id:  1 tasktime:  1 trial:  6 /41
subject_id:  1 tasktime:  1 trial:  7 /41
subject_id:  1 tasktime:  1 trial:  8 /41
subject_id:  1 tasktime:  1 trial:  9 /41
subject_id:  1 tasktime:  1 trial:  10 /41
subject_id:  1 tasktime:  1 trial:  11 /41
subject_id:  1 tasktime:  1 trial:  12 /41
subject_id:  1 tasktime:  1 trial:  13 /41
subject_id:  1 tasktime:  1 trial:  14 /41
subject_id:  1 tasktime:  1 trial:  15 /41
subject_id:  1 tasktime:  1 trial:  16 /41
subject_id:  1 tasktime:  1 trial:  17 /41
subject_id:  1 tasktime:  1 trial:  18 /41
subject_id:  1 tasktime:  1 trial:  19 /41
subject_id:  1 tasktim

KeyboardInterrupt: 

In [7]:
save_path = os.path.join('..', '..', '..', 'data', 'subjects_behavior_eeg.pkl')
subjects_behavior.to_pickle(save_path)